<a href="https://colab.research.google.com/github/Abhishek31-27/NLP_POC-s/blob/main/IMDB_and_Amazon_review_using_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data_yelp = pd.read_csv('yelp_labelled.txt', sep='\t',header=None)

In [ ]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
# Assign column names
columan_name = ['Review', 'Sentiment']
data_yelp.columns = columan_name

In [ ]:
data_yelp.shape

(1000, 2)

In [ ]:
data_amazon = pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)
data_amazon.head()
# review and sentiment
# 0-Negative, 1-Positive for positive review

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [ ]:
data_amazon.columns = columan_name
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [ ]:
data_imdb = pd.read_csv('imdb_labelled.txt',sep='\t',header=None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [ ]:
data_imdb.columns = columan_name
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [ ]:
data = data_yelp.append([data_amazon, data_imdb],ignore_index=True)
data.shape

<ipython-input-10-e5f36273f261>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data_yelp.append([data_amazon, data_imdb],ignore_index=True)


(2748, 2)

In [ ]:
data['Sentiment'].value_counts()
#Check class imbalance

1    1386
0    1362
Name: Sentiment, dtype: int64

In [ ]:
# check for null values
data.isnull().sum()

# no null values in the data

Review       0
Sentiment    0
dtype: int64

In [ ]:
x = data['Review']
y = data['Sentiment']

# Data Cleaning

# Here we will remove stopwords, punctuations
# And we will apply lemmatization
# Also creating a function to clean the data

In [ ]:
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
stopwords = list(STOP_WORDS) # list of stopwords
stopwords

['’m',
 'very',
 'few',
 'every',
 'various',
 'towards',
 'can',
 'here',
 'into',
 'for',
 'since',
 'throughout',
 'do',
 'over',
 'whoever',
 'me',
 'own',
 'less',
 'thereafter',
 'you',
 'who',
 'back',
 'should',
 'thereupon',
 'also',
 'in',
 'same',
 'could',
 'nor',
 'using',
 'an',
 'we',
 'give',
 'until',
 'was',
 'whereupon',
 'put',
 'sixty',
 'else',
 'as',
 'someone',
 'regarding',
 'ten',
 'although',
 'rather',
 'really',
 'from',
 'what',
 "'ve",
 'further',
 '‘d',
 'there',
 'were',
 'side',
 'its',
 'ours',
 'due',
 'along',
 'across',
 'once',
 'yourselves',
 'must',
 'besides',
 'third',
 'yourself',
 'formerly',
 'wherever',
 'get',
 'her',
 'now',
 'no',
 'latter',
 'not',
 'well',
 'both',
 'seems',
 'being',
 'off',
 'any',
 'sometime',
 '‘ve',
 'yet',
 'everyone',
 'and',
 'still',
 'made',
 'part',
 'too',
 'indeed',
 "'re",
 'seeming',
 'by',
 'fifteen',
 'yours',
 '’s',
 'their',
 'because',
 'he',
 'just',
 'whenever',
 'almost',
 'alone',
 'ourselves',

In [ ]:
#Fucntion for data cleaning
# if root form of that word is not pronoun then it is going to convert that into lower form
# and if that word is a proper noun, then we are directly taking lower form, because there is no lemma for proper noun
import spacy

nlp = spacy.load('en_core_web_sm')

def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)

  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [ ]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")
# stopwords and punctuations removed

['hello', 'beautiful', 'day', 'outside']

# Vectorization Feature Engineering (TF-IDF)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)
# tokenizer=text_data_cleaning, tokenization will be done according to this function

In [ ]:
classifier = LinearSVC()

In [ ]:
#Train the model
#Splitting the dataset into the Train and Test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
x_train.shape, x_test.shape
# 2198 samples in training dataset and 550 in test dataset

((2198,), (550,))

In [ ]:
x_train.head()

2572    An Italian reviewer called this "a small, grea...
526                          And it was way to expensive.
1509    As an earlier review noted, plug in this charg...
144     Nice blanket of moz over top but i feel like t...
2483    The film gives meaning to the phrase, "Never i...
Name: Review, dtype: object

In [ ]:
# it will first do vectorization and then it will do classification
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])

#Fit the x_train and y_train
clf.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7f3820b47be0>)),
                ('clf', LinearSVC())])

In [ ]:
#Predict the Test set results
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = clf.predict(x_test)

# confusion_matrix
confusion_matrix(y_test, y_pred)

array([[201,  78],
       [ 50, 221]])

In [ ]:
# classification_report
print(classification_report(y_test, y_pred))
# we are getting almost 77% accuracy

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [ ]:
accuracy_score(y_test, y_pred)
# 76% accuracy

0.7672727272727272